In [27]:
# mapping pasca to braun

In [1]:
import warnings

warnings.filterwarnings("ignore")

import os

import pandas as pd

import scvi
from scvi.model.utils import mde

import scanpy as sc

Global seed set to 0


In [2]:
DATA_DIR = "/storage/data/data/organoid_atlas/"

In [42]:
ANNOT_LABEL = "subregion_class"

In [3]:
adata_q = sc.read(os.path.join(DATA_DIR, "230605_pasca_all_v1_common_hv2k.h5ad"))

In [6]:
adata_ref = sc.read(os.path.join("/storage/data/data/organoid_atlas/jonas_query_ref_latent/braun_2022_fetal_brain_v2_common_hv2k.h5ad"))

In [ ]:
# try with orig.ident vs dataset
# include only primary/secondary

In [25]:
# subset adata_q to organoid
adata_q = adata_q[~adata_q.obs.dataset.isin(["fetal_striatum_21pcw"])]

In [26]:
adata_q.varm = dict()
adata_q.obs["batch"] = adata_q.obs["dataset"].astype(str).copy()

In [33]:
adata_ref.varm = dict()
adata_ref.obs["batch"] = adata_ref.obs["Donor"].astype(str).copy()

In [34]:
lvae = scvi.model.SCANVI.load(
        os.path.join(DATA_DIR, "query_scarches_nolabs"),
        adata=adata_ref,
    )

INFO     File /storage/data/data/organoid_atlas/query_scarches_nolabs/model.pt already downloaded                  


In [43]:
adata_q.obs[ANNOT_LABEL] = "Unknown"

In [44]:
scvi.model.SCANVI.prepare_query_anndata(adata_q, lvae)

INFO     Found 100.0% reference vars in query data.                                                                


In [45]:
vae_q = scvi.model.SCANVI.load_query_data(adata_q, lvae)

In [46]:
vae_q.train(
    max_epochs=500,
    plan_kwargs=dict(weight_decay=0.0),
)

INFO     Training for 500 epochs.                                                                                  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 500/500: 100%|█████████████████████████████████████████████████████| 500/500 [1:07:27<00:00,  8.57s/it, loss=1.19e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 500/500: 100%|█████████████████████████████████████████████████████| 500/500 [1:07:27<00:00,  8.09s/it, loss=1.19e+03, v_num=1]


In [47]:
adata_q.obs["ref"] = "query"
adata_ref.obs["ref"] = "reference"

In [48]:
adata_full = adata_q.concatenate(adata_ref)

In [52]:
adata_full.obs["batch"] = (
    adata_q.obs["batch"].tolist() + adata_ref.obs["batch"].tolist()
)

In [57]:
adata_full.obsm["X_scARCHES"] = vae_q.get_latent_representation(adata_full)
adata_full.obsm["X_umap"] = mde(adata_full.obsm["X_scARCHES"])

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [ ]:
p = sc.pl.umap(
    adata_full,
    color=["batch", "ref"],
    show=False,
)

In [62]:
p[0].figure.savefig(os.path.join(DATA_DIR,f"scanvi_full_mde.png"))

In [66]:
full_latent = pd.DataFrame(adata_full.obsm["X_scARCHES"])
full_latent.index = adata_full.obs.index

In [67]:
full_latent.to_csv(os.path.join(DATA_DIR, f"pasca_map_braun_latent.csv"))

In [68]:
vae_q.save(
    os.path.join(DATA_DIR, "pasca_map_braun_model.pt"),
    overwrite=True,
    save_anndata=False,
)